In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf

# Adjusted threshold for the new model
ANOMALY_THRESHOLD = 0.74  # Adjust as needed

# Ensure output folder exists
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")

anomaly_count = 0
output_num = 1

def mean_squared_loss(x1, x2):
    """
    Calculates the mean squared loss between two arrays.
    Returns the mean distance.
    """
    difference = x1 - x2
    sq_difference = difference ** 2
    distance = np.sqrt(sq_difference.sum())
    n_samples = np.prod(difference.shape)
    mean_distance = distance / n_samples
    return mean_distance

def combined_loss(y_true, y_pred):
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

def preprocess_frame_for_new_model(frame):
    """
    Preprocess frame for the new model architecture.
    The new model expects better normalized inputs.
    """
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    gray = gray.astype(np.float32) / 255.0
    gray = (gray - 0.5) * 2
    return gray

# Load the model
custom_objects = {'combined_loss': combined_loss}
model = load_model("../models/model.h5", custom_objects=custom_objects)

# Open the video file
cap = cv2.VideoCapture("D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4")
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames: {length}")

while cap.isOpened():
    imagedump_lis = []
    original_frame_display = None
    ret_all = True

    # Read 10 frames to create a clip
    for i in range(10):
        ret, frame = cap.read()
        if not ret:
            ret_all = False
            break

        if i == 9:
            original_frame_display = imutils.resize(frame, width=700, height=600)

        processed_frame = preprocess_frame_for_new_model(frame)
        imagedump_lis.append(processed_frame)

    if not ret_all or not imagedump_lis:
        print("End of video or issue reading frames.")
        break

    # Prepare imagedump for model prediction
    imagedump = np.array(imagedump_lis)  # Shape: (10, 232, 232)
    imagedump = np.transpose(imagedump, (1, 2, 0))  # Shape: (232, 232, 10)
    imagedump = np.expand_dims(imagedump, axis=0)  # Shape: (1, 232, 232, 10)
    imagedump = np.expand_dims(imagedump, axis=-1)  # Shape: (1, 232, 232, 10, 1)

    # Predict with the model
    output = model.predict(imagedump)

    # Calculate loss
    loss = mean_squared_loss(imagedump, output)
    loss_scaled = loss * 1000 # Scale for display
    loss_display = f"{loss_scaled:.4f}"
    print(f"Overall Loss: {loss_display}")

    if original_frame_display is not None:
        if loss_scaled > ANOMALY_THRESHOLD:
            print(f"Status: Abnormal (Score = {loss_display})")
            cv2.putText(original_frame_display, f"Abnormal: {loss_display}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.imwrite(f"output_frames/{output_num:03d}_abnormal.jpg", original_frame_display)
            anomaly_count += 1
        else:
            print(f"Status: Normal (Score = {loss_display})")
            cv2.putText(original_frame_display, f"Normal: {loss_display}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.imwrite(f"output_frames/{output_num:03d}_normal.jpg", original_frame_display)

        cv2.imshow("video", original_frame_display)
        cv2.waitKey(1)
        output_num += 1

        if output_num % 50 == 0:
            print(f"Processed {output_num} clips, found {anomaly_count} anomalies")

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f"\nProcessing complete!")
print(f"Total clips processed: {output_num-1}")
print(f"Total anomalies detected: {anomaly_count}")
if (output_num - 1) > 0:
    anomaly_rate = (anomaly_count / (output_num - 1) * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No clips were processed, cannot calculate anomaly rate.")

Total frames: 200
1/1 [==============================] - 1s 755ms/step
Overall Loss: 0.7356
Status: Normal (Score = 0.7356)
1/1 [==============================] - 0s 53ms/step
Overall Loss: 0.7315
Status: Normal (Score = 0.7315)
1/1 [==============================] - 0s 53ms/step
Overall Loss: 0.7321
Status: Normal (Score = 0.7321)
1/1 [==============================] - 0s 51ms/step
Overall Loss: 0.7351
Status: Normal (Score = 0.7351)
1/1 [==============================] - 0s 57ms/step
Overall Loss: 0.7364
Status: Normal (Score = 0.7364)
1/1 [==============================] - 0s 50ms/step
Overall Loss: 0.7362
Status: Normal (Score = 0.7362)
1/1 [==============================] - 0s 53ms/step
Overall Loss: 0.7377
Status: Normal (Score = 0.7377)
1/1 [==============================] - 0s 50ms/step
Overall Loss: 0.7430
Status: Abnormal (Score = 0.7430)
1/1 [==============================] - 0s 54ms/step
Overall Loss: 0.7502
Status: Abnormal (Score = 0.7502)
1/1 [===========================

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf

# Adjusted threshold for the new model. This threshold determines when an entire clip is considered anomalous.
ANOMALY_THRESHOLD = 0.74

# Ensure output folder exists to save annotated frames
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")

anomaly_count = 0
output_num = 1

def mean_squared_loss(x1, x2):
    """
    Calculates the mean squared loss (reconstruction error) between two arrays.
    This function is used to get an overall scalar loss for the entire clip.
    Returns the mean distance.
    """
    difference = x1 - x2
    sq_difference = difference ** 2
    distance = np.sqrt(sq_difference.sum())
    n_samples = np.prod(difference.shape)
    mean_distance = distance / n_samples
    return mean_distance

def combined_loss(y_true, y_pred):
    """
    Custom combined loss function for the Keras model.
    Assumes mean squared error is the primary component.
    """
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

def preprocess_frame_for_new_model(frame):
    """
    Preprocesses a single frame for the model.
    Resizes, converts to grayscale, normalizes pixel values to (-1, 1).
    """
    # Resize frame to the model's expected input dimensions
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    # Convert to grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to [0, 1]
    gray = gray.astype(np.float32) / 255.0
    # Further normalize to [-1, 1] as per model's expectation
    gray = (gray - 0.5) * 2
    return gray

# Load the trained Keras model
# Ensure the path to your model.h5 is correct relative to where this script is run.
custom_objects = {'combined_loss': combined_loss}
try:
    model = load_model("../models/model.h5", custom_objects=custom_objects)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'model.h5' is in the '../models/' directory.")
    exit() # Exit if model cannot be loaded

# Open the video file for processing
# Ensure the video file path is correct.
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit() # Exit if video cannot be opened

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")

while cap.isOpened():
    imagedump_lis = []
    original_frame_color = None # To store the 10th frame in its original color for display
    ret_all = True

    # Read 10 frames to create a sequence (clip) for the model
    for i in range(10):
        ret, frame = cap.read()
        if not ret:
            ret_all = False
            break

        # The 10th frame (index 9) is captured in its original color for display
        # and for overlaying the anomaly heatmap.
        if i == 9:
            original_frame_color = imutils.resize(frame, width=700, height=600)

        # Preprocess each frame for model input
        processed_frame = preprocess_frame_for_new_model(frame)
        imagedump_lis.append(processed_frame)

    # Break the loop if no more frames can be read or clip is incomplete
    if not ret_all or not imagedump_lis or original_frame_color is None:
        print("End of video or issue reading frames. Exiting.")
        break

    # Prepare the collected frames for model prediction
    # The model expects input shape (batch_size, height, width, sequence_length, channels)
    imagedump = np.array(imagedump_lis)             # Shape: (10, 232, 232)
    imagedump = np.transpose(imagedump, (1, 2, 0))  # Shape: (232, 232, 10)
    imagedump = np.expand_dims(imagedump, axis=0)   # Shape: (1, 232, 232, 10) (add batch dimension)
    imagedump = np.expand_dims(imagedump, axis=-1)  # Shape: (1, 232, 232, 10, 1) (add channel dimension)

    # Predict the reconstruction of the clip using the model
    output = model.predict(imagedump)

    # Calculate the overall reconstruction loss for the entire clip
    loss = mean_squared_loss(imagedump, output)
    loss_scaled = loss * 1000 # Scale the loss for better readability/thresholding
    loss_display = f"{loss_scaled:.4f}"
    print(f"Overall Clip Loss: {loss_display}")

    # Create a copy of the original color frame to draw on
    display_frame = original_frame_color.copy()

    # Check if the overall clip loss indicates an anomaly
    if loss_scaled > ANOMALY_THRESHOLD:
        print(f"Status: Abnormal (Score = {loss_display}) - Detecting anomalous regions...")

        # --- Anomaly Region Detection and Visualization ---
        # Extract the 10th frame (index 9) from the input and its reconstructed version
        # These frames are in the preprocessed grayscale format (values from -1 to 1)
        input_last_frame = imagedump[0, :, :, 9, 0]        # Shape: (232, 232)
        reconstructed_last_frame = output[0, :, :, 9, 0]   # Shape: (232, 232)

        # Calculate the absolute difference (reconstruction error) pixel by pixel
        diff_frame = np.abs(input_last_frame - reconstructed_last_frame)

        # Normalize diff_frame to [0, 1] for easier thresholding
        diff_frame_normalized_01 = cv2.normalize(diff_frame, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)

        # Apply a threshold to create a binary mask of high error regions.
        # This threshold is CRUCIAL for localization. Adjust this value (e.g., 0.2, 0.3, 0.4, 0.5)
        # to isolate the anomalous object. Higher values will make the mask sparser.
        # The value should be chosen such that only the anomaly's pixels are above it.
        # Start by experimenting with values like 0.2, 0.3, 0.4.
        _, anomaly_mask = cv2.threshold(diff_frame_normalized_01, 0.25, 255, cv2.THRESH_BINARY)
        anomaly_mask = anomaly_mask.astype(np.uint8) # Convert to 8-bit for colormap

        # Apply a colormap to the anomaly mask. This will color the detected anomaly regions.
        # Using COLORMAP_HOT or COLORMAP_JET on the mask.
        heatmap = cv2.applyColorMap(anomaly_mask, cv2.COLORMAP_HOT) # Changed to HOT for potentially better contrast

        # Optional: Apply a slight blur to the heatmap for smoother regions
        # Adjust kernel size (e.g., (5,5), (7,7)) as needed
        heatmap = cv2.GaussianBlur(heatmap, (5, 5), 0)

        # Resize the heatmap to match the dimensions of the original display frame
        heatmap_resized = cv2.resize(heatmap, (display_frame.shape[1], display_frame.shape[0]))

        # Overlay the heatmap on the original color frame.
        # We can use a higher alpha for the heatmap here since it's a sparse mask.
        # Adjust alpha values (e.g., 0.7 for original, 0.3 for heatmap) to control the transparency.
        display_frame = cv2.addWeighted(display_frame, 0.7, heatmap_resized, 0.3, 0)
        # --- End Anomaly Region Detection and Visualization ---

        # Add text overlay for status and save the frame
        cv2.putText(display_frame, f"Abnormal: {loss_display}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) # Red color for abnormal
        cv2.imwrite(f"output_frames/{output_num:03d}_abnormal.jpg", display_frame)
        anomaly_count += 1
    else:
        # If the clip is normal, just add text overlay and save the frame
        print(f"Status: Normal (Score = {loss_display})")
        cv2.putText(display_frame, f"Normal: {loss_display}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) # Green color for normal
        cv2.imwrite(f"output_frames/{output_num:03d}_normal.jpg", display_frame)

    # Display the processed frame in a window
    cv2.imshow("Video Playback with Anomaly Detection", display_frame)
    cv2.waitKey(1) # Wait for 1 millisecond (allows display update)

    output_num += 1

    # Print progress every 50 clips
    if output_num % 50 == 0:
        print(f"Processed {output_num-1} clips, found {anomaly_count} anomalies so far.")

    # Allow exiting the loop by pressing 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release video capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

print(f"\nProcessing complete!")
print(f"Total clips processed: {output_num-1}")
print(f"Total anomalies detected: {anomaly_count}")
if (output_num - 1) > 0:
    anomaly_rate = (anomaly_count / (output_num - 1) * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No clips were processed, cannot calculate anomaly rate.")


Model loaded successfully.
Total frames in video: 200
1/1 [==============================] - 1s 762ms/step
Overall Clip Loss: 0.7356
Status: Normal (Score = 0.7356)
1/1 [==============================] - 0s 60ms/step
Overall Clip Loss: 0.7315
Status: Normal (Score = 0.7315)
1/1 [==============================] - 0s 54ms/step
Overall Clip Loss: 0.7321
Status: Normal (Score = 0.7321)
1/1 [==============================] - 0s 52ms/step
Overall Clip Loss: 0.7351
Status: Normal (Score = 0.7351)
1/1 [==============================] - 0s 54ms/step
Overall Clip Loss: 0.7364
Status: Normal (Score = 0.7364)
1/1 [==============================] - 0s 52ms/step
Overall Clip Loss: 0.7362
Status: Normal (Score = 0.7362)
1/1 [==============================] - 0s 48ms/step
Overall Clip Loss: 0.7377
Status: Normal (Score = 0.7377)
1/1 [==============================] - 0s 52ms/step
Overall Clip Loss: 0.7430
Status: Abnormal (Score = 0.7430) - Detecting anomalous regions...
1/1 [=========================

In [ ]:
#experimetn 2 print csv

In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import imutils
import os
import tensorflow as tf
import csv # Import the csv module

# Adjusted threshold for the new model. This threshold determines when an entire clip is considered anomalous.
ANOMALY_THRESHOLD = 0.74

# Ensure output folders exist
if not os.path.exists("output_frames"):
    os.makedirs("output_frames")
if not os.path.exists("reconstructed_frames"): # Folder for reconstructed image outputs (JPEGs)
    os.makedirs("reconstructed_frames")
if not os.path.exists("output_predictions_csv"): # New folder for raw model predictions saved as CSV
    os.makedirs("output_predictions_csv")

anomaly_count = 0
output_num = 1

# Define the main CSV log file path and header
csv_file_path = "anomaly_detection_log.csv"
csv_header = [
    "clip_number",
    "overall_loss",
    "status",
    "pixel_diff_mean",
    "pixel_diff_max",
    "pixel_diff_std",
    "reconstructed_mean_pixel_value",
    "reconstructed_std_pixel_value",
    "reconstructed_min_pixel_value",
    "reconstructed_max_pixel_value"
]
csv_data = [] # List to store data rows before writing to main CSV

def mean_squared_loss(x1, x2):
    """
    Calculates the mean squared loss (reconstruction error) between two arrays.
    This function is used to get an overall scalar loss for the entire clip.
    Returns the mean distance.
    """
    difference = x1 - x2
    sq_difference = difference ** 2
    distance = np.sqrt(sq_difference.sum())
    n_samples = np.prod(difference.shape)
    mean_distance = distance / n_samples
    return mean_distance

def combined_loss(y_true, y_pred):
    """
    Custom combined loss function for the Keras model.
    Assumes mean squared error is the primary component.
    """
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return mse_loss

def preprocess_frame_for_new_model(frame):
    """
    Preprocess frame for the new model architecture.
    The new model expects better normalized inputs.
    """
    # Resize frame to the model's expected input dimensions
    frame_resized = cv2.resize(frame, (232, 232), interpolation=cv2.INTER_AREA)
    # Convert to grayscale
    gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to [0, 1]
    gray = gray.astype(np.float32) / 255.0
    # Further normalize to [-1, 1] as per model's expectation
    gray = (gray - 0.5) * 2
    return gray

# Load the trained Keras model
# Ensure the path to your model.h5 is correct relative to where this script is run.
custom_objects = {'combined_loss': combined_loss}
try:
    model = load_model("../models/model.h5", custom_objects=custom_objects)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure 'model.h5' is in the '../models/' directory.")
    exit() # Exit if model cannot be loaded

# Open the video file for processing
# Ensure the video file path is correct.
video_path = "D:\\Research Dataset\\Surveillance\\Test\\testing_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    exit() # Exit if video cannot be opened

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {length}")

while cap.isOpened():
    imagedump_lis = []
    original_frame_color = None # To store the 10th frame in its original color for display
    ret_all = True

    # Read 10 frames to create a clip
    for i in range(10):
        ret, frame = cap.read()
        if not ret:
            ret_all = False
            break

        # The 10th frame (index 9) is captured in its original color for display
        # and for overlaying the anomaly heatmap.
        if i == 9:
            original_frame_color = imutils.resize(frame, width=700, height=600)

        # Preprocess each frame for model input
        processed_frame = preprocess_frame_for_new_model(frame)
        imagedump_lis.append(processed_frame)

    # Break the loop if no more frames can be read or clip is incomplete
    if not ret_all or not imagedump_lis or original_frame_color is None:
        print("End of video or issue reading frames. Exiting.")
        break

    # Prepare the collected frames for model prediction
    # The model expects input shape (batch_size, height, width, sequence_length, channels)
    imagedump = np.array(imagedump_lis)             # Shape: (10, 232, 232)
    imagedump = np.transpose(imagedump, (1, 2, 0))  # Shape: (232, 232, 10)
    imagedump = np.expand_dims(imagedump, axis=0)   # Shape: (1, 232, 232, 10) (add batch dimension)
    imagedump = np.expand_dims(imagedump, axis=-1)  # Shape: (1, 232, 232, 10, 1) (add channel dimension)

    # Predict with the model
    output = model.predict(imagedump)

    # --- Save Model Prediction (Individual Reconstructed Frames as Images) ---
    # The output is (1, 232, 232, 10, 1). We will save each of the 10 reconstructed frames.
    for frame_idx in range(output.shape[3]): # Iterate through the 10 frames in the sequence
        reconstructed_frame = output[0, :, :, frame_idx, 0] # Get one 2D frame

        # Normalize to 0-255 and convert to 8-bit unsigned integer for saving as image
        reconstructed_frame_display = cv2.normalize(
            reconstructed_frame, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U
        )
        # Resize for better viewing if desired, or keep original model output size
        reconstructed_frame_display = cv2.resize(
            reconstructed_frame_display, (original_frame_color.shape[1], original_frame_color.shape[0])
        )
        # Save each frame with a unique name
        cv2.imwrite(f"reconstructed_frames/clip_{output_num:03d}_frame_{frame_idx+1:02d}_reconstructed.jpg", reconstructed_frame_display)
    # --- End Save Model Prediction (Images) ---

    # --- Save Raw Model Prediction as CSV (232x232 per frame) ---
    # Iterate through each of the 10 reconstructed frames in the output
    for frame_idx in range(output.shape[3]):
        # Get the 2D frame (232, 232)
        frame_to_save = output[0, :, :, frame_idx, 0]

        # Define the CSV file path for this specific frame
        per_frame_csv_path = f"output_predictions_csv/clip_{output_num:03d}_frame_{frame_idx+1:02d}_prediction.csv"

        with open(per_frame_csv_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            # Write each row of the 232x232 frame to the CSV, formatted to 5 decimal places
            for row in frame_to_save:
                formatted_row = [f"{pixel:.5f}" for pixel in row] # Format each pixel value
                csv_writer.writerow(formatted_row)

    print(f"Saved 10 reconstructed frames for clip {output_num} as 232x232 CSV files in 'output_predictions_csv/' directory.")
    # --- End Save Raw Model Prediction as CSV ---


    # Calculate the overall reconstruction loss for the entire clip
    loss = mean_squared_loss(imagedump, output)
    loss_scaled = loss * 1000 # Scale the loss for better readability/thresholding
    loss_display = f"{loss_scaled:.5f}" # Already set to 5 decimal points for print
    print(f"Overall Clip Loss: {loss_display}")

    # Create a copy of the original color frame to draw on
    display_frame = original_frame_color.copy()

    current_status = "Normal" # Default status
    pixel_diff_mean_val = 0.0
    pixel_diff_max_val = 0.0
    pixel_diff_std_val = 0.0

    # Initialize reconstructed output statistics for the main CSV log
    reconstructed_mean_pixel_value = np.mean(output)
    reconstructed_std_pixel_value = np.std(output)
    reconstructed_min_pixel_value = np.min(output)
    reconstructed_max_pixel_value = np.max(output)


    # Check if the overall clip loss indicates an anomaly
    if loss_scaled > ANOMALY_THRESHOLD:
        current_status = "Abnormal"
        print(f"Status: Abnormal (Score = {loss_display}) - Detecting anomalous regions...")

        # --- Anomaly Region Detection and Visualization ---
        # Extract the 10th frame (index 9) from the input and its reconstructed version
        # These frames are in the preprocessed grayscale format (values from -1 to 1)
        input_last_frame = imagedump[0, :, :, 9, 0]        # Shape: (232, 232)
        reconstructed_last_frame = output[0, :, :, 9, 0]   # Shape: (232, 232)

        # Calculate the absolute difference (reconstruction error) pixel by pixel
        diff_frame = np.abs(input_last_frame - reconstructed_last_frame)

        # Calculate statistics for CSV
        pixel_diff_mean_val = np.mean(diff_frame)
        pixel_diff_max_val = np.max(diff_frame)
        pixel_diff_std_val = np.std(diff_frame)

        # Normalize diff_frame to [0, 1] for easier thresholding
        diff_frame_normalized_01 = cv2.normalize(diff_frame, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)

        # Apply a threshold to create a binary mask of high error regions.
        # This threshold is CRUCIAL for localization. Adjust this value (e.g., 0.2, 0.3, 0.4, 0.5)
        # to isolate the anomalous object. Higher values will make the mask sparser.
        # The value should be chosen such that only the anomaly's pixels are above it.
        # Start by experimenting with values like 0.2, 0.3, 0.4.
        _, anomaly_mask = cv2.threshold(diff_frame_normalized_01, 0.25, 255, cv2.THRESH_BINARY)
        anomaly_mask = anomaly_mask.astype(np.uint8) # Convert to 8-bit for colormap

        # Apply a colormap to the anomaly mask. This will color the detected anomaly regions.
        # Using COLORMAP_HOT or COLORMAP_JET on the mask.
        heatmap = cv2.applyColorMap(anomaly_mask, cv2.COLORMAP_HOT) # Changed to HOT for potentially better contrast

        # Optional: Apply a slight blur to the heatmap for smoother regions
        # Adjust kernel size (e.g., (5,5), (7,7)) as needed
        heatmap = cv2.GaussianBlur(heatmap, (5, 5), 0)

        # Resize the heatmap to match the dimensions of the original display frame
        heatmap_resized = cv2.resize(heatmap, (display_frame.shape[1], display_frame.shape[0]))

        # Overlay the heatmap on the original color frame.
        # We can use a higher alpha for the heatmap here since it's a sparse mask.
        # Adjust alpha values (e.g., 0.7 for original, 0.3 for heatmap) to control the transparency.
        display_frame = cv2.addWeighted(display_frame, 0.7, heatmap_resized, 0.3, 0)
        # --- End Anomaly Region Detection and Visualization ---

        # Add text overlay for status and save the frame
        cv2.putText(display_frame, f"Abnormal: {loss_display}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) # Red color for abnormal
        cv2.imwrite(f"output_frames/{output_num:03d}_abnormal.jpg", display_frame)
        anomaly_count += 1
    else:
        # If the clip is normal, just add text overlay and save the frame
        print(f"Status: Normal (Score = {loss_display})")
        cv2.putText(display_frame, f"Normal: {loss_display}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2) # Green color for normal
        cv2.imwrite(f"output_frames/{output_num:03d}_normal.jpg", display_frame)

    # Append data for the current clip to the main CSV data list, formatted to 5 decimal points
    csv_data.append([
        output_num,
        f"{float(loss_display):.5f}", # Convert to float then format for consistency
        current_status,
        f"{pixel_diff_mean_val:.5f}",
        f"{pixel_diff_max_val:.5f}",
        f"{pixel_diff_std_val:.5f}",
        f"{reconstructed_mean_pixel_value:.5f}",
        f"{reconstructed_std_pixel_value:.5f}",
        f"{reconstructed_min_pixel_value:.5f}",
        f"{reconstructed_max_pixel_value:.5f}"
    ])

    # Display the processed frame in a window
    cv2.imshow("Video Playback with Anomaly Detection", display_frame)
    cv2.waitKey(1) # Wait for 1 millisecond (allows display update)

    output_num += 1

    # Print progress every 50 clips
    if output_num % 50 == 0:
        print(f"Processed {output_num-1} clips, found {anomaly_count} anomalies so far.")

    # Allow exiting the loop by pressing 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release video capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# --- Write main anomaly log data to CSV file after processing all clips ---
print(f"\nWriting overall anomaly log to {csv_file_path}...")
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header) # Write header
    csv_writer.writerows(csv_data) # Write all collected data rows
print("Overall anomaly log CSV file saved successfully.")
# --- End Write to CSV ---

print(f"\nProcessing complete!")
print(f"Total clips processed: {output_num-1}")
print(f"Total anomalies detected: {anomaly_count}")
if (output_num - 1) > 0:
    anomaly_rate = (anomaly_count / (output_num - 1) * 100)
    print(f"Anomaly rate: {anomaly_rate:.2f}%")
else:
    print("No clips were processed, cannot calculate anomaly rate.")


Model loaded successfully.
Total frames in video: 200
1/1 [==============================] - 1s 788ms/step
Saved 10 reconstructed frames for clip 1 as 232x232 CSV files in 'output_predictions_csv/' directory.
Overall Clip Loss: 0.73557
Status: Normal (Score = 0.73557)
1/1 [==============================] - 0s 61ms/step
Saved 10 reconstructed frames for clip 2 as 232x232 CSV files in 'output_predictions_csv/' directory.
Overall Clip Loss: 0.73151
Status: Normal (Score = 0.73151)
1/1 [==============================] - 0s 57ms/step
Saved 10 reconstructed frames for clip 3 as 232x232 CSV files in 'output_predictions_csv/' directory.
Overall Clip Loss: 0.73205
Status: Normal (Score = 0.73205)
1/1 [==============================] - 0s 53ms/step
Saved 10 reconstructed frames for clip 4 as 232x232 CSV files in 'output_predictions_csv/' directory.
Overall Clip Loss: 0.73509
Status: Normal (Score = 0.73509)
1/1 [==============================] - 0s 54ms/step
Saved 10 reconstructed frames for cli